In [1]:
%cd /content/drive/My Drive/NN_INTRO_HW

/content/drive/My Drive/NN_INTRO_HW


In [2]:
!pip install pandas

In [51]:
import pandas as pd
import re
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDRegressor
from scipy.sparse import hstack

In [3]:


weather_hist = pd.read_csv('weatherHistory.csv')
weather_hist.rename({'Formatted Date':'date', 'Temperature (C)':'temp', 'Apparent Temperature (C)':'temp_app',
                    'Precip Type':'weather_type', 'Wind Speed (km/h)':'wind_speed', 'Humidity':'humidity',
                    'Daily Summary':'summary'}, 
                    axis=1, inplace=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


Проанализируйте данные и найдите самый холодный и самый ветренный дни в Сегеде за всю имеющуюся историю.

In [5]:
min_temp = weather_hist['temp'].min()
max_wind_speed = weather_hist['wind_speed'].max()
coldest_day = weather_hist.loc[weather_hist['temp'] == min_temp]['date']
windest_day = weather_hist.loc[weather_hist['wind_speed'] == max_wind_speed]['date']
print(min_temp, coldest_day)
print(max_wind_speed, windest_day)

-21.822222222222226 54847    2012-02-10 07:00:00.000 +0100
Name: date, dtype: object
63.8526 12181    2007-01-29 13:00:00.000 +0100
Name: date, dtype: object


In [6]:
weather_hist['year'] = pd.to_datetime(weather_hist['date'], utc=True).dt.year
weather_hist['month'] = pd.to_datetime(weather_hist['date'], utc=True).dt.month

Узнайте сколько раз шел дождь летом 2014 года.

In [7]:
weather_hist[(weather_hist['weather_type'] == 'rain') & (weather_hist['year'] == 2014) & (weather_hist['month'] >= 6) & (weather_hist['month'] < 9)].shape[0]

2205

Сгруппируйтей данные по годам, месяцам и неделям (по отдельности) и найдите:

1) самый ветрянный месяц



In [8]:
weather_hist.groupby(weather_hist['month'])['wind_speed'].mean().sort_values(ascending=False)

month
3     13.415825
2     12.180815
4     11.876139
1     11.515865
12    11.094578
5     10.953186
11    10.947953
10    10.008495
7      9.637124
6      9.632453
9      9.610204
8      8.942707
Name: wind_speed, dtype: float64

2) самый влажный год

In [9]:
weather_hist.groupby(weather_hist['year'])['humidity'].mean().sort_values(ascending=False)

year
2005    0.890000
2010    0.796861
2006    0.767349
2016    0.760868
2013    0.754197
2014    0.748571
2011    0.736008
2015    0.732354
2009    0.707247
2008    0.701239
2007    0.689647
2012    0.689513
Name: humidity, dtype: float64

3) неделю с самой переменчивой погодой (там, где отклонение какого-то или нескольких признаков самое большое)

In [10]:
weather_hist = pd.read_csv('weatherHistory.csv')
weather_hist.rename({'Formatted Date':'date', 'Temperature (C)':'temp', 'Apparent Temperature (C)':'temp_app',
                    'Precip Type':'weather_type', 'Wind Speed (km/h)':'wind_speed', 'Humidity':'humidity',
                    'Daily Summary':'summary'}, 
                    axis=1, inplace=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
weather_hist['week'] = pd.to_datetime(weather_hist['date'], utc=True).dt.week
weeks = {}
for i in range(1,54):
  weeks[i] = 0
stds = weather_hist.groupby(weather_hist['week']).std()
for column in stds:
  rank = 0
  for index, row in stds[column].sort_values(ascending=False).iteritems():
    weeks[index] = weeks[index] + rank
    rank += 1
strange_week = 0
strange_rank = 1000000
for w in weeks:
  if weeks[w] < strange_rank:
    strange_week = w
    strange_rank = weeks[w]
print(strange_week)

12


Домашнее задание 2

In [11]:
data = pd.read_csv('data.csv')


Признаки, которые будем использовать для регрессии:





*   **Количество чисел в тексте** (может говорить о том, что есть подробная информация о возрасте товара или о конкретной модели, люди охотнее покупают что-то определнное и конкретное)
*   **Количество слов латиницей** (опять же, возможно, латиницей указано точное название товара)
*   **Количество личных местоимений первого лица** (если в тексте много "я", "меня", "мой" и тд, то, вероятно, человек много и нудно описывает никому не нужную историю личного использования/приобретения, вероятно, что даже дочитывать не будут)
*   **Количество аббревиатур** (также может быть обозначением бренда либо дургой конкретной информации о товаре)
*  **Количество слишком длинных слов (> 10 букв)** (если в тексте много длинных слов, скорее всего он очень заумный и сложночитаемый





In [46]:
def count_pronouns(x):
  prons = ['я',"меня", "мне", "мной", "мой", "моя", "мое", "моего", "моему", "моей", "мою" "моим", "мои", "моих", "моими"]
  x = x.lower().split()
  if x is None:
    return 0
  return len([i for i in x if i in prons])

In [47]:
def count_long_words(x):
  x = x.split()
  if x is None:
    return 0
  return len([i for i in x if len(i)>10])

In [48]:
def count_issues(x, reg):
  res = re.findall(reg, x)
  if res is None:
    return 0
  return len(res)

In [49]:

data['num_of_numbers'] = data['description'].apply(lambda x: count_issues(x, '[0-9]+[\.\,]*[0-9]*'))
data['num_of_latin_words'] = data['description'].apply(lambda x: count_issues(x,'[A-Za-z]+'))
data['num_of_pronouns'] = data['description'].apply(lambda x: count_pronouns(x))
data['num_of_abbrs'] = data['description'].apply(lambda x: count_issues(x,'[A-ZА-ЯЁ][A-ZА-ЯЁ]+'))
data['num_of_long_words'] = data['description'].apply(lambda x:count_long_words(x))

In [50]:
X = data[['num_of_numbers', 'num_of_latin_words', 'num_of_pronouns', 'num_of_abbrs', 'num_of_long_words']].values
y = data.deal_probability.values

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8)
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2636719184310224

лучше, к сожалению, не стало, но это и не удивительно, мы вообще не смотрели на данные, там, вероятно, могут быть выбросы, а возможно, просто отсутсвует корреляция с целевой переменной и эти переменнеы вообще не стоит использовать в регрессионной модели 

In [53]:


cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
X = cv.fit_transform(data.description)
y = data.deal_probability.values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
regressor = Ridge(alpha=10)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
old_res = np.sqrt(mean_squared_error(y_test, preds))
print(old_res)

0.25125158475785275


In [54]:
cvs_mf = [CountVectorizer(max_features=1000-200, min_df=5, max_df=0.4, ngram_range=(1,2)),
       CountVectorizer(max_features=1000-100, min_df=5, max_df=0.4, ngram_range=(1,2)),
       CountVectorizer(max_features=1000+100, min_df=5, max_df=0.4, ngram_range=(1,2)),
       CountVectorizer(max_features=10000+200, min_df=5, max_df=0.4, ngram_range=(1,2))]
cvs_min_df = [CountVectorizer(max_features=1000, min_df=1, max_df=0.4, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=2, max_df=0.4, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=7, max_df=0.4, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=10, max_df=0.4, ngram_range=(1,2))]
cvs_max_df = [CountVectorizer(max_features=1000, min_df=5, max_df=0.1, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=0.5, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=2.0, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=3.5, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=4.0, ngram_range=(1,2))]
cvs_ngram_range = [CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,1)),
                   CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2)),
                   CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,3)),
                   CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,4)),
                   CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,5))]
cvs = [cvs_mf, cvs_min_df, cvs_max_df, cvs_ngram_range]
for cvs_group in cvs:
  for cv in cvs_group:
    X = cv.fit_transform(data.description)
    y = data.deal_probability.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    regressor = Ridge(alpha=10)
    regressor.fit(X_train, y_train)
    preds = regressor.predict(X_test)
    res = np.sqrt(mean_squared_error(y_test, preds))  
    print(res, res <= old_res)
  print('\n')

0.25156670064082937 False
0.25142840571409236 False
0.2512676792851656 False
0.2551350294096034 False


0.25125158475785275 True
0.2512479767331968 True
0.25125158475785275 True
0.25123904679072967 True
0.2512479767331968 True


0.25156428317010743 False
0.25125158475785275 True
0.25125158475785275 True
0.25125158475785275 True
0.25125158475785275 True


0.2512640499851752 False
0.25125158475785275 True
0.25126259345294094 False
0.2512510533269011 True
0.25125690894539726 False




In [55]:

cv = CountVectorizer(max_features=1000, min_df=7, max_df=0.4, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_features=1000, min_df=5,analyzer='char', ngram_range=(3,3), max_df=0.4)

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
old_res = np.sqrt(mean_squared_error(y_test, preds))
print(old_res)

0.2496460387034647


In [56]:
regressors = [
              SGDRegressor(max_iter=50, alpha=0.00001, loss='huber'),
              SGDRegressor(max_iter=50, alpha=0.00001, loss='epsilon_insensitive'),
              SGDRegressor(max_iter=50, alpha=0.00001, loss='squared_epsilon_insensitive'),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0.2),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0.4),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0.6),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0.8),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=1),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='elasticnet'),
]
for regressor in regressors:
  regressor.fit(X_train, y_train)
  preds = regressor.predict(X_test)
  res = np.sqrt(mean_squared_error(y_test, preds))
  print(res, res <= old_res)

0.2694921854818908 False
0.25822058027974254 False
0.25392235341525554 False
0.24961863373533555 True
0.24952570165495613 True
0.24967884009502533 False
0.24975424578519956 False
0.2495411513547617 True
0.24961751108436717 True
0.24959201013552207 True


In [57]:
regressors = [
              SGDRegressor(max_iter=25, alpha=0.00001),
              SGDRegressor(max_iter=75, alpha=0.00001),
              SGDRegressor(max_iter=100, alpha=0.00001),
              SGDRegressor(max_iter=125, alpha=0.00001),
              SGDRegressor(max_iter=150, alpha=0.00001),
              SGDRegressor(max_iter=300, alpha=0.00001),
              SGDRegressor(max_iter=600, alpha=0.00001),
              SGDRegressor(max_iter=1000, alpha=0.00001),
              SGDRegressor(max_iter=50, alpha=0.000001),
              SGDRegressor(max_iter=50, alpha=0.000005),
              SGDRegressor(max_iter=50, alpha=0.00003),
              SGDRegressor(max_iter=50, alpha=0.00005),
              SGDRegressor(max_iter=50, alpha=0.0001),
              SGDRegressor(max_iter=300, alpha=0.000005, penalty='l1', l1_ratio=1),
]
for regressor in regressors:
  regressor.fit(X_train, y_train)
  preds = regressor.predict(X_test)
  res = np.sqrt(mean_squared_error(y_test, preds))
  print(res, res <= old_res)

0.24968815198251856 False
0.24976843611005917 False
0.24965810665220295 False
0.24970748113518856 False
0.24958953441283135 True
0.2497359538879487 False
0.24965643893726847 False
0.24960171469536444 True
0.24968612875329163 False
0.24964312608549494 True
0.2498938265851417 False
0.2496797390797301 False
0.24977077768879621 False
0.24971364209033803 False


Улучшить результат семинара не вышло, пробовала подбирать параметр CV, из этого ничего не вышло, хоть какое-то улучшение показало лишь изменение min_df, пробовала с хорошим вариантом min_df подбирать другие параметры, но побить результат семинара это не позволило